In [2]:
import time
import os
import math
import json
import uuid
import cv2
import numpy as np
from datetime import datetime
import atexit
from ai2thor.controller import Controller

# NLP and Transformer Imports
import spacy
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModelForQuestionAnswering

from leolani_client import LeolaniChatClient, Action

# Initialize spaCy model
nlp = spacy.load("en_core_web_sm")

# Initialize GPT-2 for text generation (question generation)
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")
gpt2_model = AutoModelForCausalLM.from_pretrained("gpt2")
gpt2_generator = pipeline('text-generation', model=gpt2_model, tokenizer=gpt2_tokenizer)

# Initialize DistilBERT for Question Answering
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad", tokenizer="distilbert-base-cased-distilled-squad")

# Mapping of room names to AI2-THOR scenes
ROOM_TO_SCENE = {
    "kitchen1": "FloorPlan4_physics",
    "kitchen2": "FloorPlan5_physics",
    "kitchen3": "FloorPlan6_physics",
    "living room": "FloorPlan1_physics",
    "bedroom": "FloorPlan2_physics",
    "bathroom": "FloorPlan3_physics"
}

SPATIAL_KEYWORDS = ["on", "under", "in", "near", "above", "below", "next to", "beside"]
ATTRIBUTE_KEYWORDS = {
    "color": ["red", "blue", "green", "yellow", "white", "black"],
    "shape": ["rectangular", "square", "circular", "round", "triangular"],
    "size": ["large", "small", "medium"]
}

print("Script started...")
print("Starting Controller...")
controller = Controller(width=800, height=600, start_unity=True, timeout=200.0)
print("Controller created.")

# Initialize metrics for interaction analysis
metrics = {
    "total_actions": 0,
    "successful_finds": 0,
    "failed_finds": 0,
    "communication_turns_human": 0,
    "communication_turns_agent": 0
}

# Initialize state for contextual awareness
state = {
    "current_floorplan": "FloorPlan1_physics",
    "found_objects": [],
    "user_descriptions": []
}

def safe_reset_scene(scene_name):
    global controller
    start_time = time.time()
    print(f"Attempting to reset scene: {scene_name}")
    try:
        controller.reset(scene_name)
        elapsed_time = time.time() - start_time
        print(f"Scene '{scene_name}' reset completed in {elapsed_time:.2f} seconds.")
    except (BrokenPipeError, TimeoutError) as e:
        print("Lost connection or timeout in AI2-THOR environment. Reinitializing controller...")
        controller.stop()
        controller = Controller(width=800, height=600, start_unity=True, timeout=200.0)
        controller.reset(scene_name)
        print(f"Scene '{scene_name}' reset completed after reinitialization.")
    except Exception as e:
        print(f"Unexpected error while resetting scene '{scene_name}': {e}")
        raise e

def calculate_distance(obj1, obj2):
    x1, y1, z1 = obj1['position']['x'], obj1['position']['y'], obj1['position']['z']
    x2, y2, z2 = obj2['position']['x'], obj2['position']['y'], obj2['position']['z']
    return math.sqrt((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2)

def is_above(obj1, obj2):
    return obj1['position']['y'] > obj2['position']['y']

def parse_description(description, visible_objects):
    doc = nlp(description)
    object_types = [obj['objectType'].lower() for obj in visible_objects]
    target_object = None
    spatial_context = []
    reference_objects = {}
    attributes = {'color': [], 'shape': [], 'size': []}

    for ent in doc.ents:
        if ent.label_ in ["COLOR", "COLOR_EXPRESSION"]:
            attributes['color'].append(ent.text.lower())
        elif ent.label_ in ["SIZE"]:
            attributes['size'].append(ent.text.lower())

    for chunk in doc.noun_chunks:
        chunk_text = chunk.text.lower()
        if chunk_text in object_types:
            if not target_object:
                target_object = chunk_text
        else:
            for obj in visible_objects:
                if chunk_text in obj['objectType'].lower() or chunk_text in obj.get('name', '').lower():
                    if chunk_text not in reference_objects:
                        reference_objects[chunk_text] = []
                    reference_objects[chunk_text].append(obj)

    for token in doc:
        if token.text.lower() in SPATIAL_KEYWORDS:
            relation = token.text.lower()
            # Assume the object being related to is the head of the preposition
            related_object = token.head.text.lower()
            spatial_context.append((relation, related_object))

    print(f"Parsed Target Object: {target_object}")
    print(f"Parsed Spatial Context: {spatial_context}")
    print(f"Parsed Reference Objects: {reference_objects}")
    print(f"Parsed Attributes: {attributes}")

    return target_object, spatial_context, reference_objects, attributes

def generate_clarifying_question(description):
    prompt = f"The user said: '{description}'. The description is ambiguous. Ask a clarifying question to get more details."
    generated = gpt2_generator(prompt, max_length=50, num_return_sequences=1)
    question = generated[0]['generated_text'].split('\n')[0]
    if not question.endswith('?'):
        question += '?'
    return question

def answer_location_question(question, context_text):
    result = qa_pipeline(question=question, context=context_text)
    return result['answer']

def _log_error(leolaniClient, message, exception):
    leolaniClient._log_error(message, exception)
    
def handle_location_query(user_input, leolaniClient):
    """
    Handles queries about the user's current location.
    """
    try:
        current_floorplan = state["current_floorplan"]
        response = f"You are currently in the {current_floorplan.replace('_physics', '').replace('FloorPlan', 'Floor Plan ')}."
        print(response)
        leolaniClient._add_utterance("System", response, {
            "current_floorplan": state["current_floorplan"],
            "visible_objects": [obj['objectType'] for obj in state["found_objects"]]
        })
        metrics["communication_turns_agent"] += 1
    except Exception as e:
        _log_error(leolaniClient, "Error handling location query", e)
        
def explore_room(leolaniClient):
    print("Exploring the room...")
    current_context = {
        "current_floorplan": state["current_floorplan"],
        "visible_objects": [obj['objectType'] for obj in state['found_objects']]
    }
    leolaniClient._add_utterance("System", "Exploring the room to detect visible objects.", current_context)
    metrics["communication_turns_agent"] += 1

    visible_objects = []
    for rotation in range(0, 360, 90):
        controller.step(action="RotateRight")
        leolaniClient._add_action("RotateRight", current_context)
        metrics["total_actions"] += 1
        visible_objects.extend(controller.last_event.metadata["objects"])

    for move in ["MoveAhead", "MoveLeft", "MoveRight", "MoveBack"]:
        controller.step(action=move)
        leolaniClient._add_action(move, current_context)
        metrics["total_actions"] += 1
        visible_objects.extend(controller.last_event.metadata["objects"])

    unique_objects = {obj['objectId']: obj for obj in visible_objects}.values()
    print("Objects found during exploration:")
    leolaniClient._add_utterance("System", "Objects found during exploration.", current_context)
    metrics["communication_turns_agent"] += 1

    for obj in unique_objects:
        obj_info = f"ID: {obj['objectId']}, Type: {obj['objectType']}, Position: {obj['position']}"
        print(obj_info)
        leolaniClient._add_utterance("System", f"Found object {obj['objectType']} with ID {obj['objectId']} at position {obj['position']}.", current_context)
        metrics["communication_turns_agent"] += 1
        state["found_objects"].append(obj)

    location_feedback = f"You are currently in the {state['current_floorplan'].replace('_physics', '').replace('FloorPlan', 'Floor Plan ')}."
    print(f"System: {location_feedback}")
    leolaniClient._add_utterance("System", location_feedback, current_context)
    metrics["communication_turns_agent"] += 1

    return list(unique_objects)

def filter_candidates_by_context(candidates, spatial_context, reference_objects, attributes, leolaniClient):
    if not spatial_context and not attributes:
        return candidates

    filtered_candidates = candidates.copy()
    
    for relation, reference in spatial_context:
        ref_candidates = reference_objects.get(reference, [])
        if not ref_candidates:
            print(f"No reference objects found for '{reference}'. Skipping spatial filtering for '{relation}'.")
            current_context = {
                "current_floorplan": state["current_floorplan"],
                "visible_objects": [obj['objectType'] for obj in state['found_objects']]
            }
            leolaniClient._add_utterance("System", f"No reference objects found for '{reference}'. Skipping spatial filtering for '{relation}'.", current_context)
            metrics["communication_turns_agent"] += 1
            continue

        print(f"Reference objects for relation '{relation}': {[r['objectType'] for r in ref_candidates]}")
        current_context = {
            "current_floorplan": state["current_floorplan"],
            "visible_objects": [obj['objectType'] for obj in state['found_objects']]
        }
        leolaniClient._add_utterance("System", f"Reference objects for relation '{relation}': {[r['objectType'] for r in ref_candidates]}.", current_context)
        metrics["communication_turns_agent"] += 1

        for ref_obj in ref_candidates:
            if relation == "on":
                filtered_candidates = [
                    c for c in filtered_candidates if is_above(c, ref_obj) and calculate_distance(c, ref_obj) <= 1.0
                ]
            elif relation == "under":
                filtered_candidates = [
                    c for c in filtered_candidates if is_above(ref_obj, c) and calculate_distance(c, ref_obj) <= 1.0
                ]
            elif relation == "near":
                filtered_candidates = [
                    c for c in filtered_candidates if calculate_distance(c, ref_obj) <= 1.0
                ]

    for attr_type, attr_values in attributes.items():
        if attr_values:
            filtered_candidates = [
                c for c in filtered_candidates if any(attr in c.get(attr_type, "").lower() for attr in attr_values)
            ]

    print(f"Filtered candidates: {[c['objectType'] for c in filtered_candidates]}")
    current_context = {
        "current_floorplan": state["current_floorplan"],
        "visible_objects": [obj['objectType'] for obj in state['found_objects']]
    }
    leolaniClient._add_utterance("System", f"Filtered candidates: {[c['objectType'] for c in filtered_candidates]}.", current_context)
    metrics["communication_turns_agent"] += 1

    return filtered_candidates

def confirm_candidates(candidates, description, visible_objects, leolaniClient):
    global state
    print("Confirming candidates...")
    current_context = {
        "current_floorplan": state["current_floorplan"],
        "visible_objects": [obj['objectType'] for obj in state['found_objects']]
    }
    leolaniClient._add_utterance("System", "Confirming candidates based on your description.", current_context)
    metrics["communication_turns_agent"] += 1

    target_object, spatial_context, reference_objects, attributes = parse_description(description, visible_objects)

    candidates = [obj for obj in visible_objects if target_object and target_object in obj['objectType'].lower()]
    if not candidates:
        print(f"No objects matching '{target_object}' were found in this floorplan.")
        leolaniClient._add_utterance("System", f"No objects matching '{target_object}' found.", current_context)
        metrics["communication_turns_agent"] += 1

        question = generate_clarifying_question(description)
        print(f"System: {question}")
        leolaniClient._add_utterance("System", question, current_context)
        metrics["communication_turns_agent"] += 1

        user_feedback = input("User: ").strip().lower()
        if user_feedback:
            leolaniClient._add_utterance("Human", user_feedback, current_context)
            metrics["communication_turns_human"] += 1
            if user_feedback == "yes":
                new_description = input("Please provide a more detailed description: ").strip()
                if new_description:
                    leolaniClient._add_utterance("Human", new_description, current_context)
                    metrics["communication_turns_human"] += 1
                    state["user_descriptions"].append(new_description)
                    return False, True 
        return False, True  

    candidates = filter_candidates_by_context(candidates, spatial_context, reference_objects, attributes, leolaniClient)
    if not candidates:
        print(f"No objects matching the description '{description}' were found.")
        leolaniClient._add_utterance("System", f"No objects matching the description '{description}' found.", current_context)
        metrics["communication_turns_agent"] += 1

        question = generate_clarifying_question(description)
        print(f"System: {question}")
        leolaniClient._add_utterance("System", question, current_context)
        metrics["communication_turns_agent"] += 1

        user_feedback = input("User: ").strip().lower()
        if user_feedback:
            leolaniClient._add_utterance("Human", user_feedback, current_context)
            metrics["communication_turns_human"] += 1
            if user_feedback == "yes":
                new_description = input("Please provide a more detailed description: ").strip()
                if new_description:
                    leolaniClient._add_utterance("Human", new_description, current_context)
                    metrics["communication_turns_human"] += 1
                    state["user_descriptions"].append(new_description)
                    return False, True 
        return False, True  

    # Iterate through all candidates and confirm with the user
    for obj in candidates:
        if orient_and_display_object(obj, leolaniClient):
            print(f"Object ID: {obj['objectId']}, Type: {obj['objectType']}")
            leolaniClient._add_action(f"Displayed object {obj['objectType']} with ID {obj['objectId']}.", current_context)
            metrics["communication_turns_agent"] += 1

            user_feedback = input(f"Is this the correct '{obj['objectType']}'? (yes/no): ").strip().lower()
            if not user_feedback:
                print("Feedback cannot be empty. Please respond with 'yes' or 'no'.")
                leolaniClient._add_utterance("System", "User provided empty feedback.", current_context)
                metrics["communication_turns_agent"] += 1
                continue

            leolaniClient._add_utterance("Human", user_feedback, current_context)
            metrics["communication_turns_human"] += 1

            if user_feedback == "yes":
                save_object_details(obj)
                leolaniClient._add_utterance("System", "Object found and confirmed!", current_context)
                metrics["successful_finds"] += 1
                return True, False  # End the program
            elif user_feedback == "no":
                ask_for_detailed_description = input("Do you want to give a more detailed description? (yes/no): ").strip().lower()
                if ask_for_detailed_description == "yes":
                    new_description = input("Please provide a more detailed description: ").strip()
                    if new_description:
                        leolaniClient._add_utterance("Human", new_description, current_context)
                        metrics["communication_turns_human"] += 1
                        state["user_descriptions"].append(new_description)
                        return False, True 
           
            return False, True  

    print(f"All possible '{target_object}' objects in this floorplan have been shown.")
    leolaniClient._add_utterance("System", f"All candidates for '{target_object}' have been presented.", current_context)
    metrics["communication_turns_agent"] += 1

    detailed_description = input("Do you want to give a more detailed description? (yes/no): ").strip().lower()
    if not detailed_description:
        print("Response cannot be empty. Please respond with 'yes' or 'no'.")
        leolaniClient._add_utterance("System", "User provided empty feedback.", current_context)
        metrics["communication_turns_agent"] += 1
        return False, True

    leolaniClient._add_utterance("Human", detailed_description, current_context)
    metrics["communication_turns_human"] += 1

    if detailed_description == "yes":
        new_description = input("Please provide a more detailed description: ").strip()
        if new_description:
            leolaniClient._add_utterance("Human", new_description, current_context)
            metrics["communication_turns_human"] += 1
            state["user_descriptions"].append(new_description)
            return False, True 

    print("Moving to the next floorplan with the current description...")
    leolaniClient._add_utterance("System", "Moving to the next floorplan with current description.", current_context)
    metrics["communication_turns_agent"] += 1
    return False, True 

def orient_and_display_object(obj, leolaniClient):
    print(f"Orienting towards object: {obj['objectType']} at {obj['position']}")
    current_context = {
        "current_floorplan": state["current_floorplan"],
        "visible_objects": [obj['objectType'] for obj in state['found_objects']]
    }
    leolaniClient._add_utterance("System", f"Orienting towards the {obj['objectType']} located at position {obj['position']}.", current_context)
    metrics["communication_turns_agent"] += 1

    try:
        controller.step(
            action="TeleportFull",
            position={
                "x": obj['position']['x'],
                "y": obj['position']['y'] + 1.0,
                "z": obj['position']['z'] - 0.5
            },
            rotation={"x": 0.0, "y": 0.0, "z": 0.0},
            horizon=30.0,
            standing=True
        )
        metrics["total_actions"] += 1
        time.sleep(0.5)

        frame = controller.last_event.frame
        if frame is not None:
            image_path = display_image(frame, obj["objectType"])
            if image_path:
                leolaniClient._add_image(obj["objectType"], obj["objectType"], obj["position"], image_path, current_context)
                leolaniClient._add_utterance("System", f"I have displayed an image of the {obj['objectType']} at {image_path}.", current_context)
                metrics["communication_turns_agent"] += 1
            return image_path is not None
        else:
            print("Failed to capture frame: No frame data available.")
            leolaniClient._add_utterance("System", "Failed to capture frame: No visual data available.", current_context)
            metrics["communication_turns_agent"] += 1
            return False
    except Exception as e:
        error_message = f"Error during object orientation or frame capture: {str(e)}"
        print(error_message)
        leolaniClient._add_utterance("System", error_message, current_context)
        metrics["communication_turns_agent"] += 1
        return False

def display_image(frame, object_type):
    """
    Saves the frame as an image file and returns the file path.
    """
    try:
        output_dir = "./images"
        os.makedirs(output_dir, exist_ok=True)
        filename = os.path.join(output_dir, f"{object_type}_{int(time.time())}.png")
        cv2.imwrite(filename, frame)
        print(f"Image saved as {filename}. Please open it to view the object.")
        return filename
    except Exception as e:
        print(f"Failed to save image: {e}")
        return None

def save_object_details(obj):
    output_dir = "./object_details"
    os.makedirs(output_dir, exist_ok=True)
    filename = os.path.join(output_dir, "found_object.txt")
    with open(filename, "w") as f:
        f.write(f"Object ID: {obj['objectId']}\n")
        f.write(f"Object Type: {obj['objectType']}\n")
        f.write(f"Position: {obj['position']}\n")
    print(f"Object details saved to {filename}.")

def main_loop():
    global state
    global controller 

    emissor_path = "./emissor"
    if not os.path.exists(emissor_path):
        os.makedirs(emissor_path, exist_ok=True)

    leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent="Ai2Thor", human="Human")

    print("Starting object search. Please describe the object.")
    current_context = {
        "current_floorplan": state["current_floorplan"],
        "visible_objects": [obj['objectType'] for obj in state['found_objects']]
    }
    leolaniClient._add_utterance("System", "Starting object search.", current_context)
    metrics["communication_turns_agent"] += 1

    while True:
        try:
            user_input = input("\nDescribe the object you're looking for (or type 'exit' to quit): ").strip()
            if not user_input:
                print("Description cannot be empty. Please provide a valid input.")
                leolaniClient._add_utterance("System", "User provided empty description.", current_context)
                metrics["communication_turns_agent"] += 1
                continue

            if "where am i" in user_input.lower() or "where am I" in user_input.lower():
                handle_location_query(user_input, leolaniClient)
                continue

            current_context = {
                "current_floorplan": state["current_floorplan"],
                "visible_objects": [obj['objectType'] for obj in state['found_objects']]
            }
            leolaniClient._add_utterance("Human", user_input, current_context)
            metrics["communication_turns_human"] += 1
            state["user_descriptions"].append(user_input)

            if user_input.lower() in ["exit", "quit"]:
                print("Exiting the program.")
                leolaniClient._add_utterance("System", "Exiting the program.", current_context)
                metrics["communication_turns_agent"] += 1
                break

            scenes_to_search = ROOM_TO_SCENE.values()

            for scene_name in scenes_to_search:
                safe_reset_scene(scene_name)
                state["current_floorplan"] = scene_name
                visible_objects = explore_room(leolaniClient)

                success, move_to_next = confirm_candidates([], user_input, visible_objects, leolaniClient)
                if success:
                    print("Object found!")
                    leolaniClient._add_utterance("System", "Object found successfully.", current_context)
                    metrics["successful_finds"] += 1
                    break
                if move_to_next:
                    print("Moving to the next floorplan...")
                    leolaniClient._add_utterance("System", "Moving to the next floorplan.", current_context)
                    metrics["failed_finds"] += 1
                    continue  

            leolaniClient._save_scenario()

        except Exception as e:
            leolaniClient._log_error("Unhandled exception in main loop", e)
            print(f"An error occurred: {str(e)}")

    print("DEBUG: Explicitly calling _write_interactions()")
    leolaniClient._write_interactions()

    metrics_file = os.path.join(emissor_path, 'metrics_summary.json')
    try:
        with open(metrics_file, 'w') as f:
            json.dump(metrics, f, indent=4)
        print(f"Metrics saved to {metrics_file}.")
    except Exception as e:
        print(f"Failed to save metrics: {e}")
        leolaniClient._log_error("Error saving metrics", e)

if __name__ == "__main__":
    main_loop()


Device set to use cpu
Device set to use cpu


Script started...
Starting Controller...
Controller created.
Starting object search. Please describe the object.
INFO:LeolaniChatClientLogger:{"speaker": "System", "utterance": "Starting object search.", "action": "utterance", "timestamp": "2024-12-23T21:11:40.365914Z", "context": {"current_floorplan": "FloorPlan1_physics", "visible_objects": []}}



Describe the object you're looking for (or type 'exit' to quit):  apple on the countertop


INFO:LeolaniChatClientLogger:{"speaker": "Human", "utterance": "apple on the countertop", "action": "utterance", "timestamp": "2024-12-23T21:11:54.022141Z", "context": {"current_floorplan": "FloorPlan1_physics", "visible_objects": []}}
Attempting to reset scene: FloorPlan4_physics
Scene 'FloorPlan4_physics' reset completed in 21.91 seconds.
Exploring the room...
INFO:LeolaniChatClientLogger:{"speaker": "System", "utterance": "Exploring the room to detect visible objects.", "action": "utterance", "timestamp": "2024-12-23T21:12:15.930231Z", "context": {"current_floorplan": "FloorPlan4_physics", "visible_objects": []}}
INFO:LeolaniChatClientLogger:{"speaker": "System", "utterance": "RotateRight", "action": "agent_action", "timestamp": "2024-12-23T21:12:18.210230Z", "context": {"current_floorplan": "FloorPlan4_physics", "visible_objects": []}}
INFO:LeolaniChatClientLogger:{"speaker": "System", "utterance": "RotateRight", "action": "agent_action", "timestamp": "2024-12-23T21:12:23.988706Z",

Is this the correct 'Apple'? (yes/no):  no


...

Moving to the next floorplan...
INFO:LeolaniChatClientLogger:{"speaker": "System", "utterance": "Moving to the next floorplan.", "action": "utterance", "timestamp": "2024-12-23T21:13:19.658128Z", "context": {"current_floorplan": "FloorPlan1_physics", "visible_objects": []}}
Attempting to reset scene: FloorPlan5_physics
Scene 'FloorPlan5_physics' reset completed in 12.73 seconds.
Exploring the room...
INFO:LeolaniChatClientLogger:{"speaker": "System", "utterance": "Exploring the room to detect visible objects.", "action": "utterance", "timestamp": "2024-12-23T21:13:32.383681Z", "context": {"current_floorplan": "FloorPlan5_physics", "visible_objects": ["Apple", "Bowl", "Bread", "ButterKnife", "Cabinet", "Cabinet", "CoffeeMachine", "CounterTop", "CounterTop", "CounterTop", "Cup", "DiningTable", "DishSponge", "Drawer", "Drawer", "Drawer", "Drawer", "Drawer", "Drawer", "Egg", "Faucet", "Floor", "Fork", "Fridge", "GarbageCan", "HousePlant", "Knife", "Ladle", "Lettuce", "LightSwitch", "Microw

Is this the correct 'Apple'? (yes/no):  yes


INFO:LeolaniChatClientLogger:{"speaker": "Human", "utterance": "yes", "action": "utterance", "timestamp": "2024-12-23T21:14:13.118559Z", "context": {"current_floorplan": "FloorPlan5_physics", "visible_objects": ["Apple", "Bowl", "Bread", "ButterKnife", "Cabinet", "Cabinet", "CoffeeMachine", "CounterTop", "CounterTop", "CounterTop", "Cup", "DiningTable", "DishSponge", "Drawer", "Drawer", "Drawer", "Drawer", "Drawer", "Drawer", "Egg", "Faucet", "Floor", "Fork", "Fridge", "GarbageCan", "HousePlant", "Knife", "Ladle", "Lettuce", "LightSwitch", "Microwave", "Mug", "Pan", "PepperShaker", "Plate", "Pot", "Potato", "SaltShaker", "Sink", "SinkBasin", "SoapBottle", "Spatula", "Spoon", "StoveBurner", "StoveBurner", "StoveBurner", "StoveBurner", "StoveKnob", "StoveKnob", "StoveKnob", "StoveKnob", "Toaster", "Tomato", "Window", "Window", "Apple", "Bowl", "Bread", "ButterKnife", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "


Describe the object you're looking for (or type 'exit' to quit):  exit


INFO:LeolaniChatClientLogger:{"speaker": "Human", "utterance": "exit", "action": "utterance", "timestamp": "2024-12-23T21:14:18.976822Z", "context": {"current_floorplan": "FloorPlan5_physics", "visible_objects": ["Apple", "Bowl", "Bread", "ButterKnife", "Cabinet", "Cabinet", "CoffeeMachine", "CounterTop", "CounterTop", "CounterTop", "Cup", "DiningTable", "DishSponge", "Drawer", "Drawer", "Drawer", "Drawer", "Drawer", "Drawer", "Egg", "Faucet", "Floor", "Fork", "Fridge", "GarbageCan", "HousePlant", "Knife", "Ladle", "Lettuce", "LightSwitch", "Microwave", "Mug", "Pan", "PepperShaker", "Plate", "Pot", "Potato", "SaltShaker", "Sink", "SinkBasin", "SoapBottle", "Spatula", "Spoon", "StoveBurner", "StoveBurner", "StoveBurner", "StoveBurner", "StoveKnob", "StoveKnob", "StoveKnob", "StoveKnob", "Toaster", "Tomato", "Window", "Window", "Apple", "Bowl", "Bread", "ButterKnife", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", "Cabinet", 